In [3]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2      

## Load the training data into feature matrix, class labels, and event ids:

In [16]:
from proj1_helpers import *

#Pensez à retirer 'fermeli' du path
DATA_TRAIN_PATH = '/home/fermeli/ML_course/projects/project1/data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)


def clean_data(tX):
    tX[tX == -999] = np.NaN
    mean = np.nanmean(tX,axis=0)
    inds = np.where(np.isnan(tX))    
    tX[inds]= np.take(mean, inds[1])
    
    std= np.std(tX,axis=0)
    newMean = np.nanmean(tX,axis=0)
    
    return (tX-newMean)/std

tX = clean_data(tX)
print("finished")



[ 1. -1. -1. ...  1. -1. -1.]
finished


## Do your thing crazy machine learning thing here :) ...

In [59]:
def calculate_mse(e):
   return 1/2*np.mean(e**2)


def compute_gradient(y, tx, w):
    err = y - tx.dot(w)
    grad = -tx.T.dot(err) / len(err)
    return grad, err



############################### Linear regression with gradient descent################################   

def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    ws = [initial_w]
    y= y.reshape(y.shape[0],1)
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        grad, err = compute_gradient(y, tx, w)
        loss = calculate_mse(err)
        w = w - gamma * grad
        ws.append(w)
        losses.append(loss)
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}, w1={w1}, w1={w1}".format(
        bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    return losses, ws
#least_squares_GD(y, tX, np.full((30,1),0.00001), 100, 0.001)


######## least squares ##################################################################################
def least_square(y,tx):
    s= tx.T.dot(tx)
    t = tx.T.dot(y)
    return np.linalg.solve(s, t)


################################# Linear regression with SGD ###########################################




def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]


def stochastic_gradient_descent(
        y, tx, initial_w, batch_size, max_iters, gamma):
    """Stochastic gradient descent."""
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    y= y.reshape(y.shape[0],1)
    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y, tx, batch_size=batch_size, num_batches=1):
            # compute a stochastic gradient and loss
            grad, _ = compute_gradient(y_batch, tx_batch, w)
            # update w through the stochastic gradient update
            w = w - gamma * grad
            # calculate loss
            e = y - tx.dot(w)
            loss = calculate_mse(e)
            # store w and loss
            ws.append(w)
            losses.append(loss)

        print("SGD({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    return losses, ws


def least_squares_SGD(y, tx, initial_w,max_iters, gamma):
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO: implement stochastic gradient descent.
    # ***************************************************
    batch_size = 1
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        # ***************************************************
        # INSERT YOUR CODE HERE
        # TODO: compute gradient and loss
        # ***************************************************
        
        B = tx[np.random.choice(batch_size):]
        
        grad = compute_gradient(y,B,w)
        loss = compute_loss(y,tx,w)
        # ***************************************************
        # INSERT YOUR CODE HERE
        # TODO: update w by gradient
        # ***************************************************
        w = w - gamma * grad
        # store w and loss
        ws.append(w)
        losses.append(loss)
        
    return losses, ws

#stochastic_gradient_descent(y, tX, np.full((30,1),0.00001),4, 100, 0.00001)

################################## Ridge regression ##############################################


def ridge_regression_solve(y, tx, lambda_):
    aI = 2 * tx.shape[0] * lambda_ * np.identity(tx.shape[1])
    a = tx.T.dot(tx) + aI
    b = tx.T.dot(y)
    return np.linalg.solve(a, b)

def ridge_regression(y, tx, lambda_ ):
    y= y.reshape(y.shape[0],1)
    w = ridge_regression_solve(y, tx, lambda_)
    err = y - tx.dot(w)
    rmse = np.sqrt(2 * calculate_mse(err))
    return rmse, w
    

        
#lambdas = np.logspace(-5, 0, 15)
#for ind, lambda_ in enumerate(lambdas):
 #   ridge_regression(y, tX,lambda_)

    
    
################################## Logisitic regression ##################################

def accuracy(labels_gt, labels_pred):
    """ Computes accuracy.
    
    Args:
        labels_gt (np.array): GT labels of shape (N, ).
        labels_pred (np.array): Predicted labels of shape (N, ).
        
    Returns:
        float: Accuracy, in range [0, 1].
    """
    
    labels_gt = np.squeeze(labels_gt)
    
    result = 0.0
    
    for i in range(len(labels_gt)):
        if labels_gt[i] == labels_pred[i]:
            result += 1.0
    
    return result/(0.0 + len(labels_gt))

def sigmoid(t):
    t[t<-10] = -10
    sigmoid = 1/(1 +np.exp(-t)) 
    sigmoid[sigmoid == 1] = 0.9999
    return sigmoid

def calculate_loss(y, tx, w):
    
    sig = sigmoid(tx.dot(w))
    
    cost =  - (1-y).T.dot(np.log(1-sig)) - y.T.dot(np.log(sig))
    return cost
    

def calculate_gradient_LR(y, tx, w):
    
    return tx.T.dot(sigmoid(tx.dot(w))- y)
                               
    
def learning_by_gradient_descent(y, tx, w, gamma):
    loss = calculate_loss(y, tx, w)
    grad = calculate_gradient_LR(y,tx,w)
    w = w- gamma * grad
    
    return loss, w

def logistic_regression(y, tx, initial_w, max_iters, gamma):
    y[y == -1] = 0 #because we have to have values between 0 and 1
    threshold = 1e-8
    losses = []
    y= y.reshape(y.shape[0],1)
    w = initial_w
    # start the logistic regression
    for iter in range(max_iters):
        # get loss and update w.
        loss, w = learning_by_gradient_descent(y, tx, w, gamma)
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        #print(loss)
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
            
    return loss, w

def classify_with_threshold(threshold,tX,w):
    predictions = sigmoid(tX @ w)
    labels = []
    for pred in predictions:
        labels.append(0 if pred < threshold else 1)
        
    return labels

################################## Logisitic regression with REgula##################################
def penalized_logistic_regression(y, tx, w, lambda_):
    num_samples = y.shape[0]
    
    loss = calculate_loss(y, tx, w) + lambda_ * np.squeeze(w.T.dot(w))
    gradient = calculate_gradient_LR(y, tx, w) + 2 * lambda_ * w
    return loss, gradient

def learning_by_penalized_gradient(y, tx, w, gamma, lambda_):
   
    loss, gradient = penalized_logistic_regression(y, tx, w, lambda_)
    w -= gamma * gradient
    return loss, w

def reg_logistic_regression(y, tx, lambda_ , initial_w, max_iters, gamma):
    # init parameters
    threshold = 1e-8
    losses = []
    w = initial_w
    y = y.reshape(y.shape[0],1)
    # start the logistic regression
    for iter in range(max_iters):
        # get loss and update w.
        loss, w = learning_by_penalized_gradient(y, tx, w, gamma, lambda_)
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    return losses,w 

#reg_logistic_regression(y, tX, 0.1, np.zeros((tX.shape[1], 1)), 10000, 0.01)

################################## Functions to optimize the model ##################################

def split_data(x, y, ratio, seed=1):
    """
    split the dataset based on the split ratio. If ratio is 0.8 
    you will have 80% of your data set dedicated to training 
    and the rest dedicated to testing
    """
    # set seed
    np.random.seed(seed)
    # ***************************************************
    # INSERT YOUR CODE HERE
    # split the data based on the given ratio: TODO
    # ***************************************************
    indices = np.arange(len(y))
    training_indices = np.random.choice(indices, (int(ratio * len(y))))
    test_indices = np.setdiff1d(indices,training_indices)
    training_x = np.take(x,training_indices)
    training_y = np.take(y,training_indices)
    test_x = np.take(x,test_indices)
    test_y = np.take(y,test_indices)
    
    
    return training_x,test_x,training_y,test_y




def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree."""
    poly = np.ones((len(x), 1))
    for deg in range(1, degree+1):
        poly = np.c_[poly, np.power(x, deg)]
    return poly

def find_best_gamma():
    poly = build_poly(tX,4)
    gammas = np.linspace(0, 1, 15)
    losses = []
    for ind, gamma in enumerate(gammas):
        loss,_ = logistic_regression(y, poly, np.zeros((poly.shape[1], 1)), 1000, gamma)
        losses.append((loss,gamma))
        print()
   
    return min(losses, key = lambda t: t[0])

print("starting log reg")

#print(find_best_gamma())

best_ac = 0
best_deg = 0
#for degree in range(2,10):
#    poly = build_poly(tX,degree)
#    _,w = np.squeeze(logistic_regression(y, poly, np.zeros((poly.shape[1], 1)), 1000, 0.01))
#   labels = classify_with_threshold(0.5,poly,w)
#    acc = accuracy(y,labels)
#    print("accuracy with degree " + str(degree) + " is : " + str(acc))
#    if acc > best_ac:
#        best_ac = acc
#        best_deg = degree

#thresh = np.linspace(0,1,15)
#print(thresh)

#find_best_gamma()

#poly = build_poly(tX,4)
#_,w = np.squeeze(logistic_regression(y, poly, np.zeros((poly.shape[1], 1)), 1000, 0.5))
#labels = classify_with_threshold(0.55,poly,w)
#acc = accuracy(y,labels)
#print(acc)

#for t in thresh:

    #poly = build_poly(tX,4)
    #_,w = np.squeeze(logistic_regression(y, poly, np.zeros((poly.shape[1], 1)), 1000, 0.01))
    #labels = classify_with_threshold(t,poly,w)
    #acc = accuracy(y,labels)
    #print("with threshold " + str(t) + " accuracy of " + str(acc))
    #if acc > best_ac:
    #    best_ac = acc
    #    best_deg = degree
    

    
#print("best acc is " + str(best_ac) + " obtained with degree " + str(best_deg))

#lambdas = np.linspace(0.0,1,15)

#best_lambda = 0.0
#best_acc = 0.0

#poly = build_poly(tX,4)

#for i in lambdas:
#    loss,w = ridge_regression(y, poly, i)
#    y_pred = predict_labels(w,poly)
#    acc = accuracy(y,y_pred)
#    if acc > best_acc:
#        best_acc = acc
#        best_lambda = i
        
#print("best acc is " + str(best_acc) + " obtained with lambda " + str(best_lambda))

best_lambda = 0
best_acc = 0
best_degree = 0
for degree in range(8,15):
    poly = build_poly(tX,degree)
    loss,w = ridge_regression(y, poly, best_lambda)
    y_pred = predict_labels(w,poly)
    acc = accuracy(y,y_pred)
    if acc > best_acc:
        best_acc = acc
        best_degree = degree
        
print("best acc is " + str(best_acc) + " obtained with degree " + str(best_degree))



#print(y_pred)
#print(y)



#print(acc)


starting log reg
best acc is 0.816332 obtained with degree 10


## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)